<a href="https://colab.research.google.com/github/achih030/learning-portfolio/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai==0.28
!pip install google-cloud-vision

In [ ]:
import os
import io
from google.colab import files
from google.cloud import vision

# 設置OpenAI API密鑰
openai.api_key = 'open_api_key'

# 設置Google Cloud Vision API服務賬戶密鑰文件路徑
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "google-cloud.json"

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def extract_texts_from_files(file_paths):
    extracted_texts = []
    client = vision.ImageAnnotatorClient()

    for file_path in file_paths:
        with io.open(file_path, 'rb') as file:
            content = file.read()

        if file_path.endswith(('.png', '.jpg', '.jpeg')):
            image = vision.Image(content=content)
            response = client.text_detection(image=image)
            texts = response.text_annotations
            extracted_texts.extend([text.description for text in texts])

    return extracted_texts

def summarize_texts(time_experience_reflections):
    combined_text = "\n\n".join(time_experience_reflections)
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "你是一個幫助生成台灣高中要做的學習歷程的助手。"},
            {"role": "user", "content": f"根據以下內容生成一段自然大約500至1000個中文字的流暢的台灣高中的學習歷程與心得：\n\n{combined_text}\n"}
        ],
        temperature=0.9
    )
    summary = response['choices'][0]['message']['content'].strip()
    return summary

def delete_files(file_paths):
    for file_path in file_paths:
        os.remove(file_path)
        print(f"已刪除文件：{file_path}")

def upload_files():
    print("請上傳要處理的獎狀圖片文件...(點擊CANCEL結束上傳)")
    file_paths = []
    time_experience_reflections = []  # 存儲每個文件的時間、歷程和心得
    while True:
        uploaded_file = files.upload()
        if not uploaded_file:
            break
        for name, content in uploaded_file.items():
            if allowed_file(name):
                with open(name, 'wb') as file:
                    file.write(content)
                file_paths.append(name)
                # 讓使用者輸入時間、歷程和心得
                time = input("請輸入參加時間：")
                experience = input("請輸入想補充的過程：")
                reflection = input("請輸入想補充的心得：")
                # 將輸入的內容寫入列表中
                time_experience_reflections.append(f"時間：{time}\n\n歷程：{experience}\n\n心得：{reflection}")
            else:
                print(f"文件 {name} 的格式不符合要求，請重新上傳 PNG、JPG 或 JPEG 格式的文件。")

    return file_paths, time_experience_reflections

if __name__ == "__main__":
    print("歡迎使用學習歷程總結工具！")
    file_paths, time_experience_reflections = upload_files()
    if not file_paths:
        print("未上傳任何文件。")
    else:
        print(f"已上傳{len(file_paths)}個文件。")
        print("正在提取文本...")
        extracted_texts = extract_texts_from_files(file_paths)
        print("正在總結學習歷程...")
        summary = summarize_texts(time_experience_reflections)
        print("\n學習歷程總結：")
        print(summary)
        print("正在刪除已上傳的文件...")
        delete_files(file_paths)
